# hyper-parameters tuning of an machine learning model (Random Forest)
<a href="https://www.linkedin.com/in/ouassim-adnane/" >Ouassim Adnane </a> <b>v1=24/03/2019 v2=20/07/2020</b>

<img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F848555%2F034d45ec284f87d0641309564207efc9%2FYJZT-16CM-7-2CM-Car-Sticker-Vinyl-Decal-Tuning-Turbo-Snail-Turbo-Funny-Black-Silver-C10.jpg_640x640.jpg?generation=1595196705118028&alt=media">

Hyperparameter optimization or tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm. A hyperparameter is a parameter whose value is used to control the learning process. By contrast, the values of other parameters (typically node weights) are learned.

### About Data 

In this competition we have data about small mobile robot driving over different floor surfaces. We need to predict the floor type based on robot's sensor data.

## Table of content <a class="anchor" id="toc"></a>

<div style="background: #f9f9f9 none repeat scroll 0 0;border: 1px solid #aaa;display: table;font-size: 95%;margin-bottom: 1em;padding: 20px;width: 400px;">
<h2>Contents</h2>
<ul style="font-weight: 700;text-align: left;list-style: outside none none !important;">
<li style="font-size: 20px;list-style: outside none none !important;"><a href="#g">1 Grid Searchg</a></li>
<li style="font-size: 20px;list-style: outside none none !important;"><a href="#r">2 Randomized Search</a></li>
<li style="font-size: 20px;list-style: outside none none !important;"><a href="#b">3 Baysian optimization</a></li>
<li style="font-size: 20px;list-style: outside none none !important;"><a href="#t">4 Tree of Parzen Estimators (TPE)</a></li>
<li style="font-size: 20px;list-style: outside none none !important;"><a href="#h">5 Hyperband </a></li>
    <li style="font-size: 20px;list-style: outside none none !important;"><a href="#ga">6 Genetic Algorithm</a></li>

</ul>
</div>

<h2 style="color:red">If you enjoyed this work or you found it helpful , an upvotes would be very much appreciated  :-)</h2>

## Imports 

In [ ]:
import time
import numpy as np
import pandas as pd
import timeit
import random 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split
    
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from hyperopt import  tpe,hp,fmin



import warnings
warnings.simplefilter('ignore')

In [ ]:
train_df = pd.read_csv('../input/career-con-2019/X_train.csv')
test_df = pd.read_csv('../input/career-con-2019/X_test.csv')
target = pd.read_csv('../input/career-con-2019/y_train.csv')
sub = pd.read_csv('../input/career-con-2019/sample_submission.csv')

# train = pd.read_csv("../input/titanic/train.csv")

 ### Some quick feature engineering based of this <a href="https://www.kaggle.com/prashantkikani/help-humanity-by-helping-robots">kernel </a>

In [ ]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z
def feature_eng(data):
    data['total_angular_velocity'] = (data['angular_velocity_X'] ** 2 + data['angular_velocity_Y'] ** 2 + data['angular_velocity_Z'] ** 2) ** 0.5
    data['total_linear_acceleration'] = (data['linear_acceleration_X'] ** 2 + data['linear_acceleration_Y'] ** 2 + data['linear_acceleration_Z'] ** 2) ** 0.5
    data['acc_vs_vel'] = data['total_linear_acceleration'] / data['total_angular_velocity']

    x, y, z, w = data['orientation_X'].tolist(), data['orientation_Y'].tolist(), data['orientation_Z'].tolist(), data['orientation_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)

    data['euler_x'] = nx
    data['euler_y'] = ny
    data['euler_z'] = nz

    data['total_angle'] = (data['euler_x'] ** 2 + data['euler_y'] ** 2 + data['euler_z'] ** 2) ** 0.5
    data['angle_vs_acc'] = data['total_angle'] / data['total_linear_acceleration']
    data['angle_vs_vel'] = data['total_angle'] / data['total_angular_velocity']
    df = pd.DataFrame()
    
    for col in data.columns:
        if col in ['row_id', 'series_id', 'measurement_number']:
            continue
        df[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        df[col + '_min'] = data.groupby(['series_id'])[col].min()
        df[col + '_max'] = data.groupby(['series_id'])[col].max()
        df[col + '_std'] = data.groupby(['series_id'])[col].std()
        df[col + '_max_to_min'] = df[col + '_max'] / df[col + '_min']

        df[col + '_abs_max'] = data.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby('series_id')[col].apply(lambda x: np.min(np.abs(x)))
        
    return df


In [ ]:
new_training_data = feature_eng(train_df)
new_test_data = feature_eng(test_df)

In [ ]:
new_training_data.fillna(0, inplace = True)
new_test_data.fillna(0, inplace = True)
new_training_data.replace(-np.inf, 0, inplace = True)
new_training_data.replace(np.inf, 0, inplace = True)
new_test_data.replace(-np.inf, 0, inplace = True)
new_test_data.replace(np.inf, 0, inplace = True)

In [ ]:
le = LabelEncoder()
target = le.fit_transform(target['surface'])

# Grid Search <a class="anchor" id="g"></a>
<a href="#toc"><img src= "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Circle-icons-arrow-up.svg/1200px-Circle-icons-arrow-up.svg.png" style="width:20px;hight:20px;float:left" >Back to the table of contents</a>

Grid search is arguably the most basic hyperparameter tuning method. With this technique, we simply build a model for each possible combination of all of the hyperparameter values provided, evaluating each model, and selecting the architecture which produces the best results

<img src="https://cambridgecoding.files.wordpress.com/2016/03/gridsearch_cv.png" style="height:300px;"/>

In [ ]:
rf_param_grid = {
                 'max_depth' : list(range(8,100,10)),
                 'n_estimators': list(range(50,2000,100)),
                 'min_samples_split': [2,6,8],
                 'min_samples_leaf': [2,6,8],
                 'bootstrap': [True, False]
                 }


In [ ]:
m = RandomForestClassifier(n_jobs=-1)

In [ ]:
m_r = GridSearchCV(param_grid=rf_param_grid, estimator = m, scoring = "accuracy", cv = 4)

In [ ]:
# %time m_r.fit(new_training_data, target)

estimated time GridSearch  3^4⋅2=484 different parameter combinations. cross-validation is 4-fold cv so the above code should train your model 484⋅4=1944 times.GridSearch runs parallel on your processors, we have 4 processors available, each processor should fit the model 1944/4=484 times. Now,on average my model takes 10sec to train, I'm estimating around 484⋅10/60=80.6min training time.
Grid Search is accurate but take a long time  to find the best hyperparameters 

# Randomized Search <a class="anchor" id="r"></a>
<a href="#toc"><img src= "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Circle-icons-arrow-up.svg/1200px-Circle-icons-arrow-up.svg.png" style="width:20px;hight:20px;float:left" >Back to the table of contents</a>

Another way to search through hyperparameter space to find optima is via randomized search. In randomized search, you sample HP values a certain number of times from some distribution which you prespecify in advance. So unlike grid search, in which you specify particular numbers to combinatorially try out, you instead specify distributions that cover the HP space you want to explore. For example, you might specify a standard normal distribution over an HP if you think reasonable values are roughly centered around 0, or a uniform distribution over some range if you think values within that range are about as likely to be “good”. In randomized search, you also specify a n_iter parameter, which acts as a computational budget, controlling how many different parameter settings are tried out in total.

<img src="https://raw.githubusercontent.com/nslatysheva/data_science_blogging/master/expanding_ML_toolkit/expanding_toolkit.jpg" style="height:500px;width:50%;"/>

In [ ]:
m_r = RandomizedSearchCV(param_distributions=rf_param_grid, estimator = m, scoring = "accuracy", cv = 4,n_jobs=-1,n_iter=10)

In [ ]:
%time m_r.fit(new_training_data,target)

In [ ]:
m_r.best_params_

In [ ]:
for param, score in zip(m_r.cv_results_['params'], m_r.cv_results_['mean_test_score']):
    print(param, score)

In [ ]:
rf_bp = m_r.best_params_

In [ ]:
rf_classifier=RandomForestClassifier(n_estimators=rf_bp["n_estimators"],
                                     min_samples_split=rf_bp['min_samples_split'],
                                     min_samples_leaf=rf_bp['min_samples_leaf'],
                                     max_depth=rf_bp['max_depth'],
                                     bootstrap=rf_bp['bootstrap'])

In [ ]:
rf_classifier.fit(new_training_data,target)

In [ ]:
y_pred = rf_classifier.predict(new_test_data)

In [ ]:
sub['surface'] = le.inverse_transform(y_pred)
sub.to_csv('random_f_rs.csv', index=False)

# Baysian optimization <a class="anchor" id="b"></a>
<a href="#toc"><img src= "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Circle-icons-arrow-up.svg/1200px-Circle-icons-arrow-up.svg.png" style="width:20px;hight:20px;float:left" >Back to the table of contents</a>

Bayesian approaches, in contrast to random or grid search, keep track of past evaluation results which they use to form a probabilistic model mapping hyperparameters to a probability of a score on the objective function:<br>
<b style="margin-left:425px">P(Score | Hyperparameters)</b>


In the literature, this model is called a “surrogate” for the objective function and is represented as p(y | x). The surrogate is much easier to optimize than the objective function and Bayesian methods work by finding the next set of hyperparameters to evaluate on the actual objective function by selecting hyperparameters that perform best on the surrogate function. In other words:<br>
<ol>
<li>Build a surrogate probability model of the objective function</li>
<li>Find the hyperparameters that perform best on the surrogate</li>
<li>Apply these hyperparameters to the true objective function</li>
<li>Update the surrogate model incorporating the new results</li>
<li>Repeat steps 2–4 until max iterations or time is reached</li>
</ol>

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true">

A great kernel about  bayesian optimization in more depth https://www.kaggle.com/artgor/bayesian-optimization-for-robots

In [ ]:
n_fold = 4
folds = StratifiedKFold(n_splits=n_fold, shuffle=True)

In [ ]:
def randomforest_evaluate(**params):
    
    params['n_estimators'] = int(round(params['n_estimators'],0))
    params['min_samples_split'] = int(round(params['min_samples_split'],0))
    params['min_samples_leaf'] = int(round(params['min_samples_leaf'],0))
    params['bootstrap'] = int(round(params['bootstrap'],0))
        
    
    test_pred_proba = np.zeros((new_training_data.shape[0],9))
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(new_training_data, target)):
        X_train, X_valid = new_training_data.iloc[train_idx], new_training_data.iloc[valid_idx]
        y_train, y_valid = target[train_idx], target[valid_idx]
        
        model = RandomForestClassifier(**params,n_jobs=-1)
        
        model.fit(X_train, y_train)
        
        y_pred_valid = model.predict_proba(new_training_data)
       
        test_pred_proba += y_pred_valid / folds.n_splits
        
  
 
    return accuracy_score(target, test_pred_proba.argmax(1))

In [ ]:
rf_param_grid = {
                 'max_depth' : (8,100),
                 'n_estimators': (50,2000),
                 'min_samples_split': (2,10),
                 'min_samples_leaf': (2, 10),
                 'bootstrap': (True, False),
                 }

rf_b_o = BayesianOptimization(randomforest_evaluate, rf_param_grid)
%time rf_b_o.maximize(init_points=5, n_iter=20)

In [ ]:
for i,n in rf_b_o.max["params"].items():
    print(i,int(round(n)))

In [ ]:
model=RandomForestClassifier(n_estimators=int(round(rf_b_o.max["params"]["n_estimators"],0)),max_depth=int(round(rf_b_o.max["params"]["max_depth"])),
                             min_samples_leaf=int(round(rf_b_o.max["params"]["min_samples_leaf"])),min_samples_split=int(round(rf_b_o.max["params"]["min_samples_split"])))

model.fit(new_training_data,target)
y_pred = model.predict(new_test_data)
sub['surface'] = le.inverse_transform(y_pred)
sub.to_csv('random_f_bo.csv', index=False)

# Tree of Parzen Estimators (TPE) <a class="anchor" id="t"></a>
<a href="#toc"><img src= "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Circle-icons-arrow-up.svg/1200px-Circle-icons-arrow-up.svg.png" style="width:20px;hight:20px;float:left" >Back to the table of contents</a>

<img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F848555%2F8bb65d72e78f31dd0840129abc4882b7%2Ftpe.PNG?generation=1595247115729068&alt=media">

<img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F848555%2F1fa3ac0ef82c94bcf584ac09f402a833%2F1_m1JYCHHhbmn6TCHcJIklZA.gif?generation=1595247193531898&alt=media">

### Research Paper [TPE](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)

### [Hyperopt](https://github.com/hyperopt/hyperopt) github repo 

In [ ]:
def randomforest_evaluate(params):
    print(params)
    params['n_estimators'] = int(round(params['n_estimators'],0))
    params['min_samples_split'] = int(round(params['min_samples_split'],0))
    params['min_samples_leaf'] = int(round(params['min_samples_leaf'],0))
    params['bootstrap'] = int(round(params['bootstrap'],0))
        
    test_pred_proba = np.zeros((new_training_data.shape[0],9))
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(new_training_data, target)):
        X_train, X_valid = new_training_data.iloc[train_idx], new_training_data.iloc[valid_idx]
        y_train, y_valid = target[train_idx], target[valid_idx]
        
        model = RandomForestClassifier(**params,n_jobs=-1)
        
        model.fit(X_train, y_train)
        
        y_pred_valid = model.predict_proba(new_training_data)
        test_pred_proba += y_pred_valid / folds.n_splits
        
  
    return accuracy_score(target, test_pred_proba.argmax(1))

# define a search space


space = {
    
        'max_depth': hp.uniform('max_depth', 8,100),
        'n_estimators': hp.uniform('n_estimators', 50, 2000),
        'min_samples_split' : hp.choice('min_samples_split',[2,3,6,9]),
        'min_samples_leaf' : hp.choice('min_samples_leaf', [2,3,6,9]),
        'bootstrap': hp.choice('bootstrap', [False,True]),
        
    }

# minimize the objective over the space

%time best = fmin(randomforest_evaluate, space, algo=tpe.suggest, max_evals=25)

if best["min_samples_split"] <=1:
    best["min_samples_split"] = 2
    
if best["min_samples_split"] <=1:
    best["min_samples_leaf"] = 2
    
print("Best Params:")
print(best)

In [ ]:
model=RandomForestClassifier(n_estimators=int(round(best["n_estimators"],0)),max_depth=int(round(best["max_depth"],0)),
                             min_samples_leaf=int(round(best["min_samples_leaf"])),min_samples_split=int(round(best["min_samples_split"])))

model.fit(new_training_data,target)
y_pred = model.predict(new_test_data)
sub['surface'] = le.inverse_transform(y_pred)
sub.to_csv('random_f_tpe.csv', index=False)

# Hyperband Bandit-Based Approach to Hyperparameter Optimization <a class="anchor" id="h"></a>
<a href="#toc"><img src= "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Circle-icons-arrow-up.svg/1200px-Circle-icons-arrow-up.svg.png" style="width:20px;hight:20px;float:left" >Back to the table of contents</a>

Performance of machine learning algorithms depends critically on identifying a good set of hyperparameters. While recent approaches use Bayesian optimization to adaptively select configurations, we focus on speeding up random search through adaptive resource allocation and early-stopping. We formulate hyperparameter optimization as a pure-exploration non-stochastic infinite-armed bandit problem where a predefined resource like iterations, data samples, or features is allocated to randomly sampled configurations. We introduce a novel algorithm, Hyperband, for this framework and analyze its theoretical properties, providing several desirable guarantees. Furthermore, we compare Hyperband with popular Bayesian optimization methods on a suite of hyperparameter optimization problems. We observe that Hyperband can provide over an order-of-magnitude speedup over our competitor set on a variety of deep-learning and kernel-based learning problems. © 2018 Lisha Li, Kevin Jamieson, Giulia DeSalvo, Afshin Rostamizadeh and Ameet Talwalkar.

<img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F848555%2F90a14c59c51968f01468f646fbb86d2a%2Fhb.PNG?generation=1595248059112556&alt=media">

### Research Paper [HyperBand](https://arxiv.org/pdf/1603.06560.pdf)

### [Github](https://github.com/thuijskens/scikit-hyperband) Repo

In [ ]:
!git clone https://github.com/thuijskens/scikit-hyperband.git 2>/dev/null 1>/dev/null

In [ ]:
!cp -r scikit-hyperband/* .

In [ ]:
!python setup.py install 2>/dev/null 1>/dev/null

In [ ]:
from hyperband import HyperbandSearchCV

from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelBinarizer

model = RandomForestClassifier()

param_dist = {
    
    'max_depth': sp_randint(8,100),
    'n_estimators': sp_randint(50,2000),
    'min_samples_split' : [2,3,6,9,10],
    'min_samples_leaf' : [2,3,6,9,10],
    'bootstrap': [True, False]
}

y = LabelBinarizer().fit_transform(target)

search = HyperbandSearchCV(model, param_dist, 
                           resource_param='n_estimators',
                           scoring='accuracy')
%time search.fit(new_training_data, y)
print(search.best_params_)

In [ ]:
best=search.best_params_

In [ ]:
model=RandomForestClassifier(n_estimators=int(round(best["n_estimators"],0)),max_depth=int(round(best["max_depth"],0)),
                             min_samples_leaf=best["min_samples_leaf"],min_samples_split=best["min_samples_split"])

model.fit(new_training_data,target)
y_pred = model.predict(new_test_data)
sub['surface'] = le.inverse_transform(y_pred)
sub.to_csv('random_f_hb.csv', index=False)


# Hyperparameter tuning using genetic algorithm <a class="anchor" id="h"></a>
<a href="#ga"><img src= "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Circle-icons-arrow-up.svg/1200px-Circle-icons-arrow-up.svg.png" style="width:20px;hight:20px;float:left" >Back to the table of contents</a>

<img src="https://image.slidesharecdn.com/final-160831053134/95/genetic-algorithm-1-638.jpg?cb=1472621627">

Genetic algorithms are stochastic search algorithms which act on a population of possible solutions. They are loosely based on the mechanics of population genetics and selection. The potential solutions are encoded as ‘genes’ — strings of characters from some alphabet. New solutions can be produced by ‘mutating’ members of the current population, and by ‘mating’ two solutions together to form a new solution. The better solutions are selected to breed and mutate and the worse ones are discarded. They are probabilistic search methods; this means that the states which they explore are not determined solely by the properties of the problems. A random process helps to guide the search. Genetic algorithms are used in artificial intelligence like other search algorithms are used in artificial intelligence — to search a space of potential solutions to find one which solves the problem

<img src="https://cdn.educba.com/academy/wp-content/uploads/2019/09/What-is-Genetic-Algorithm.png">

## All the Acknowledgment to [Mohit Jain github repo](http://https://github.com/mjain72/Hyperparameter-tuning-in-XGBoost-using-genetic-algorithm) for the code 

### Genetic Algorithm [youtube playlist](http://https://www.youtube.com/playlist?list=PLRqwX-V7Uu6bJM3VgzjNV5YxVxUwzALHV) (the Coding train)


In [ ]:
def initilialize_poplulation(numberOfParents):
    max_depth = np.empty([numberOfParents, 1], dtype = np.uint8)
    n_estimators = np.empty([numberOfParents, 1], dtype = np.uint8)
    min_samples_split = np.empty([numberOfParents, 1])
    min_samples_leaf = np.empty([numberOfParents, 1])
    bootstrap = np.empty([numberOfParents, 1])
    
    for i in range(numberOfParents):
        #print(i)
        max_depth[i] = int(random.randrange(8, 100,step=1))
        n_estimators[i] = int(random.randrange(50, 2000, step = 100))
        min_samples_split[i] = int(random.randrange(2, 10, step= 1))
        min_samples_leaf[i] = int(random.randrange(2, 10))
        bootstrap[i] = int(random.randint(0, 1))
       
    
    population = np.concatenate((max_depth, n_estimators, min_samples_split, min_samples_leaf, bootstrap), axis= 1)
    return population

In [ ]:
def fitness_accscore(y_true, y_pred):
    fitness = round(accuracy_score(y_true, y_pred), 4)
    return fitness


#train the data annd find fitness score
def train_population(population, train,y_train, test, y_test):
    fScore = []
    for i in range(population.shape[0]):

        param_dist = {

                'max_depth': int(population[i][0]),
                'n_estimators': int(population[i][1]),
                'min_samples_split' : int(population[i][2]),
                'min_samples_leaf' : int(population[i][3]),
                'bootstrap': int(population[i][4])
            }
        #print(param_dist)           
        model = RandomForestClassifier(**param_dist)
        model.fit(train,y_train)
        preds = model.predict(test)
        fScore.append(fitness_accscore(y_test, preds))
    return fScore

In [ ]:
#select parents for mating
def new_parents_selection(population, fitness, numParents):
    selectedParents = np.empty((numParents, population.shape[1])) #create an array to store fittest parents
    
    #find the top best performing parents
    for parentId in range(numParents):
        bestFitnessId = np.where(fitness == np.max(fitness))
        bestFitnessId  = bestFitnessId[0][0]
        selectedParents[parentId, :] = population[bestFitnessId, :]
        fitness[bestFitnessId] = -1 #set this value to negative, in case of F1-score, so this parent is not selected again
    return selectedParents

In [ ]:
'''
Mate these parents to create children having parameters from these parents (we are using uniform crossover method)
'''
def crossover_uniform(parents, childrenSize):
    
    crossoverPointIndex = np.arange(0, np.uint8(childrenSize[1]), 1, dtype= np.uint8) #get all the index
    crossoverPointIndex1 = np.random.randint(0, np.uint8(childrenSize[1]), np.uint8(childrenSize[1]/2)) # select half  of the indexes randomly
    crossoverPointIndex2 = np.array(list(set(crossoverPointIndex) - set(crossoverPointIndex1))) #select leftover indexes
    
    children = np.empty(childrenSize)
    
    '''
    Create child by choosing parameters from two parents selected using new_parent_selection function. The parameter values
    will be picked from the indexes, which were randomly selected above. 
    '''
    for i in range(childrenSize[0]):
        
        #find parent 1 index 
        parent1_index = i%parents.shape[0]
        #find parent 2 index
        parent2_index = (i+1)%parents.shape[0]
        #insert parameters based on random selected indexes in parent 1
        children[i, crossoverPointIndex1] = parents[parent1_index, crossoverPointIndex1]
        #insert parameters based on random selected indexes in parent 1
        children[i, crossoverPointIndex2] = parents[parent2_index, crossoverPointIndex2]
    return children

In [ ]:
def mutation(crossover, numberOfParameters):
    #Define minimum and maximum values allowed for each parameter
    minMaxValue = np.zeros((numberOfParameters, 2))

    minMaxValue[0:] = [8, 100] #min/max max depth
    minMaxValue[1, :] = [50, 2000] #min/max n_estimator
    minMaxValue[2, :] = [2, 10] #min/max min_samples_split
    minMaxValue[3, :] = [2, 10] #min/max min_samples_leaf
    minMaxValue[4, :] = [0, 1] #min/max boostrap

 
    # Mutation changes a single gene in each offspring randomly.
    mutationValue = 0
    parameterSelect = np.random.randint(0,5, 1)
    #print(parameterSelect)
    if parameterSelect == 0: #depth
        mutationValue = np.random.randint(-50, 50, 1)
    if parameterSelect == 1: #n_estimator
        mutationValue = np.random.randint(-200, 200, 1)
    if parameterSelect == 2: #min_samples_split
        mutationValue = np.random.randint(-10, 10, 1)
    if parameterSelect == 3: #min_samples_leaf
        mutationValue = np.random.randint(-10, 10, 1)
    if parameterSelect == 4: #boostrap
        mutationValue = 0

  
    #indtroduce mutation by changing one parameter, and set to max or min if it goes out of range
    for idx in range(crossover.shape[0]):
        crossover[idx, parameterSelect] = crossover[idx, parameterSelect] + mutationValue
        if(crossover[idx, parameterSelect] > minMaxValue[parameterSelect, 1]):
            crossover[idx, parameterSelect] = minMaxValue[parameterSelect, 1]
        if(crossover[idx, parameterSelect] < minMaxValue[parameterSelect, 0]):
            crossover[idx, parameterSelect] = minMaxValue[parameterSelect, 0]    
    return crossover

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(new_training_data, target, test_size = 0.20, random_state = 97)


In [ ]:
start = timeit.default_timer()
numberOfParents = 8 #number of parents to start
numberOfParentsMating = 4 #number of parents that will mate
numberOfParameters = 5 #number of parameters that will be optimized
numberOfGenerations = 4 #number of genration that will be created
#define the population size
populationSize = (numberOfParents, numberOfParameters)
#initialize the population with randomly generated parameters
population = initilialize_poplulation(numberOfParents)
#print(population)
#define an array to store the fitness  hitory
fitnessHistory = np.empty([numberOfGenerations+1, numberOfParents])
#define an array to store the value of each parameter for each parent and generation
populationHistory = np.empty([(numberOfGenerations+1)*numberOfParents, numberOfParameters])
#insert the value of initial parameters in history
populationHistory[0:numberOfParents, :] = population
for generation in range(numberOfGenerations):
    print("This is number %s generation" % (generation))
    
    #train the dataset and obtain fitness
    fitnessValue = train_population(population=population, train=X_train,y_train=y_train, test=X_test, y_test=y_test)
    fitnessHistory[generation, :] = fitnessValue
    
    #best score in the current iteration
    print('Best accuracy score in the this iteration = {}'.format(np.max(fitnessHistory[generation, :])))
    #survival of the fittest - take the top parents, based on the fitness value and number of parents needed to be selected
    parents = new_parents_selection(population=population, fitness=fitnessValue, numParents=numberOfParentsMating)
    
    #mate these parents to create children having parameters from these parents (we are using uniform crossover)
    children = crossover_uniform(parents=parents, childrenSize=(populationSize[0] - parents.shape[0], numberOfParameters))
    
    #add mutation to create genetic diversity
    children_mutated = mutation(children, numberOfParameters)
    
    '''
    We will create new population, which will contain parents that where selected previously based on the
    fitness score and rest of them  will be children
    '''
    population[0:parents.shape[0], :] = parents #fittest parents
    population[parents.shape[0]:, :] = children_mutated #children
    
    populationHistory[(generation+1)*numberOfParents : (generation+1)*numberOfParents+ numberOfParents , :] = population #srore parent information


stop = timeit.default_timer()
print('Execution time: ', (stop - start)/60)  

In [ ]:
#Best solution from the final iteration
fitness = train_population(population=population, train=X_train,y_train=y_train, test=X_test, y_test=y_test)
fitnessHistory[generation+1, :] = fitness
#index of the best solution
bestFitnessIndex = np.where(fitness == np.max(fitness))[0][0]
#Best fitness
print("Best fitness is =", fitness[bestFitnessIndex])
#Best parameters
print("Best parameters are:")
print('Max_depth', int(population[bestFitnessIndex][0]))
print('n_estimators', int(population[bestFitnessIndex][1]))
print('min_samples_split', int(population[bestFitnessIndex][2])) 
print('min_samples_leaf', int(population[bestFitnessIndex][3]))
print('bootstrap', int(population[bestFitnessIndex][4]))


In [ ]:
model=RandomForestClassifier(n_estimators=int(population[bestFitnessIndex][1]),max_depth=int(population[bestFitnessIndex][0]),
                             min_samples_leaf=int(population[bestFitnessIndex][3]),min_samples_split=int(population[bestFitnessIndex][2]))

model.fit(new_training_data,target)
y_pred = model.predict(new_test_data)
sub['surface'] = le.inverse_transform(y_pred)
sub.to_csv('random_f_ga.csv', index=False)


<a href="#toc"><img  src="https://za.heytv.org/wp-content/uploads/2019/08/AGF-l79DYZtk_pSyfWgIP3D-3yi8YN6ZeWO0E8tyLgs800-c-k-c0xffffffff-no-rj-mo.jpeg" style="height: 300px"/></a>